In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from matplotlib.colors import ListedColormap

#классификация
from sklearn.linear_model import LogisticRegression #логистическая регрессия
from sklearn.neural_network import MLPClassifier #нейронка
from sklearn.naive_bayes import GaussianNB #наивный байес
from sklearn.neighbors import KNeighborsClassifier # k ближайших соседей

#регрессия
from sklearn.linear_model import LinearRegression #линейная регрессия
from sklearn.svm import SVC #метод опорных векторов


#Оценка качества в задачах классификации
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


#Оценка качества в задачах регрессии
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

from sklearn import preprocessing

In [2]:
#Задача: вычисление значения параметра code на основании остальных параметров.
train_data = pd.read_csv("Набор_1_музеи_России.csv",delimiter=';')#Меняем разделитель на ;


In [3]:
#выбираем рандомное число
my_code = "АнтоновДмитрий"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [4]:
# Определим размер валидационной выборки
val_size = round(0.05*len(train_data))
print(val_size)

114


In [5]:
# Создадим обучающую и валидационную выборки
random_state = my_seed
train, val = train_test_split(train_data, 
                              test_size=val_size, 
                              random_state=random_state)
print(len(train), len(val))

2160 114


In [6]:
print(train)

                                                   name  base_fond  \
773                        Угутский краеведческий музей       5837   
444                 Воткинский музей истории и культуры      14730   
1547  Музей истории и культурного наследия Агрызског...       4072   
2103                     Порховский краеведческий музей        623   
946                            Троицкий городской музей        583   
...                                                 ...        ...   
1013                Объед.мемориальный музей Ю.Гагарина      29246   
1579            Музейно-литературный центр "Дом поэтов"         40   
537     Музей истории села Елхово им.Халиуллы Ахметшина       9879   
859   Азовский ист.-археологический и палеонтологиче...     225189   
262                 Омский музей театрального искусства       8290   

      science_fond     ams  n_buildings  n_days  guests  employees  
773            606    6443            1     247     1.6         12  
444           1610   

In [7]:
print(val)

                                                   name  base_fond  \
745             Ковдорский районный краеведческий музей       4078   
2017                  Красногорский краеведческий музей        541   
69             Бабушкинский районный исторический музей        769   
970   Краеведческий музей "Поморье" Кемского муницип...       7620   
483                       Акшинский краеведческий музей      11195   
...                                                 ...        ...   
44    Литературно-художественный музей М.И. и А.И.Цв...      14166   
1163             Музей-заповедник "Костромская слобода"       2847   
2052              Перевозский музейно-выставочный центр       2662   
944                 Щелковский ист.-краеведческий музей      10528   
232                          Музей-усадьба В.И.Сурикова        520   

      science_fond   ams  n_buildings  n_days  guests  employees  
745           2813     0            1     256    13.0         10  
2017            20     0 

In [8]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
name=['name']
num_columns = ['base_fond','science_fond','ams','n_buildings','n_days','employees']
ord_columns = ['guests']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')

ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['base_fond', 'science_fond', 'ams',
                                  'n_buildings', 'n_days', 'employees'])])

In [9]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_val = pd.DataFrame(ct.transform(val))

# даем название столбцам
column_names = num_columns +['name']+ ord_columns
sc_train.columns = column_names
sc_val.columns = column_names

sc_train

,base_fond,science_fond,ams,n_buildings,n_days,employees,name,guests
0,0.001255,0.001511,0.00181,0.0,0.65407,0.004326,Угутский краеведческий музей,1.6
1,0.003168,0.004013,0.0,0.0,0.80814,0.004719,Воткинский музей истории и культуры,23.0
2,0.000875,0.001087,0.0,0.0,0.648256,0.002359,Музей истории и культурного наследия Агрызског...,3.4
3,0.000133,0.011509,0.0,0.012605,0.997093,0.007865,Порховский краеведческий музей,28.8
4,0.000125,0.006065,0.0,0.0,0.65407,0.002359,Троицкий городской музей,6.1
...,...,...,...,...,...,...,...,...
2155,0.006291,0.119704,0.006698,0.037815,0.843023,0.045615,Объед.мемориальный музей Ю.Гагарина,58.6
2156,0.000008,0.000603,0.000011,0.0,0.65407,0.000393,"Музейно-литературный центр ""Дом поэтов""",1.7
2157,0.002125,0.00851,0.00003,0.0,0.718023,0.00118,Музей истории села Елхово им.Халиуллы Ахметшина,20.0
2158,0.048448,0.429905,0.08344,0.042017,0.994186,0.084153,Азовский ист.-археологический и палеонтологиче...,203.0


In [10]:
# В качестве входных параметров используем первые 9 числовых параметров,
# в качестве выходного - десятый числовой параметр.
x_train = sc_train[num_columns]
x_val = sc_val[num_columns]

y_train = (sc_train[ord_columns].values).flatten()
y_val = (sc_val[ord_columns].values).flatten()

In [11]:
y_train=np.array(y_train, dtype = np.int64)
y_val=np.array(y_val, dtype = np.int64)

In [12]:
#Создаем модели
models = [GaussianNB(),
              LogisticRegression(penalty='l2', tol=0, C=10, 
                                 solver='lbfgs', max_iter=10000)]

In [13]:
#Обучаем модели
for model in models:
    print(model)
    model.fit(x_train, y_train)

GaussianNB()
LogisticRegression(C=10, max_iter=10000, tol=0)


In [14]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
AccuracyScore = []
PrecisionScore = []
RecallScore = []

for model in models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    AS = accuracy_score(y_val, val_pred)
    ps = precision_score(y_val, val_pred, average='weighted',zero_division = 1)
    rs = recall_score(y_val, val_pred, average='weighted',zero_division = 1)
    
    f1s.append(f1)
    AccuracyScore.append(AS)
    PrecisionScore.append(ps)
    RecallScore.append(rs)
    print(model, '\t', f1,'\t','f1')
    print(model, '\t', AS,'\t','AccuracyScore')
    print(model, '\t', ps,'\t','PrecisionScore')
    print(model, '\t', rs,'\t','RecallScore')

GaussianNB() 	 0.015090357472628942 	 f1
GaussianNB() 	 0.03508771929824561 	 AccuracyScore
GaussianNB() 	 0.4226415915719821 	 PrecisionScore
GaussianNB() 	 0.03508771929824561 	 RecallScore
LogisticRegression(C=10, max_iter=10000, tol=0) 	 0.011895836457239968 	 f1
LogisticRegression(C=10, max_iter=10000, tol=0) 	 0.043859649122807015 	 AccuracyScore
LogisticRegression(C=10, max_iter=10000, tol=0) 	 0.5782593739250086 	 PrecisionScore
LogisticRegression(C=10, max_iter=10000, tol=0) 	 0.043859649122807015 	 RecallScore


In [15]:
#выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = models[i_max]
print(best_c_model,'\t','f1')


GaussianNB() 	 f1


In [16]:
i_max = AccuracyScore.index(max(AccuracyScore))
best_c_model = models[i_max]
print(best_c_model,'\t','AccuracyScore')

LogisticRegression(C=10, max_iter=10000, tol=0) 	 AccuracyScore


In [17]:
i_max = PrecisionScore.index(max(PrecisionScore))
best_c_model = models[i_max]
print(best_c_model,'\t','PrecisionScore')

LogisticRegression(C=10, max_iter=10000, tol=0) 	 PrecisionScore


In [18]:
i_max = RecallScore.index(max(RecallScore))
best_c_model = models[i_max]
print(best_c_model,'\t','RecallScore')

LogisticRegression(C=10, max_iter=10000, tol=0) 	 RecallScore
